In [ ]:
import math
from risq import *

### Data (control group)

In [ ]:
# The distribution of breast cancer cases per 100,000 population in Japan in 2018,
# according to [13, Fig. 1B]. Note: we substract the lag time of 5 years.
control_group_distribution = [(age - 5, cases / 100_000) for (age, cases) in [
    (22, 5),
    (27, 10),
    (32, 25),
    (37, 80),
    (42, 180),
    (47, 260),
    (52, 250),
    (57, 230),
    (62, 245),
    (67, 255),
    (72, 240),
    (77, 215),
    (82, 175),
    (87, 120),
]]

### Parameter fitting

In [ ]:
def loss_two_state_model(prob_mutate: float, prob_spread: float, *, verbose: bool = False) -> float:
    # Convert probabilities using math.erf (for better optimization behaviour)
    prob_mutate = (1.0 + math.erf(prob_mutate)) / 2.0
    prob_spread = (1.0 + math.erf(prob_spread)) / 2.0

    # Create model and simulatation
    model = create_two_state_model(
        prob_mutate=prob_mutate,
        prob_spread=prob_spread,
    )
    simulation = SingleCellMethod(model)

    # Assumption: number of cells
    num_cells = 1_000_000

    # Compute loss as total square difference of probabilities
    loss = 0.0
    prev_prob_cdf = 0.0
    for age, prob in control_group_distribution:
        prob_cdf = compute_cancer_probability(
            method=simulation,
            num_cells=num_cells,
            time=age * 12,  # 1 time step = 1 month
            state_cancer=1  # C
        )
        prob_est = prob_cdf - prev_prob_cdf
        loss += (prob_est - prob) ** 2

        if verbose:
            print(f"model({age}) = {prob_cdf - prev_prob_cdf} (expected {prob})")

        prev_prob_cdf = prob_cdf

    if verbose:
        print()
        print(f"prob_mutate = {prob_mutate}")
        print(f"prob_spread = {prob_spread}")
        print()
        print(f"loss = {loss}")

    return loss

In [ ]:
def loss_six_mutations_model(
    prob_mutate: float,
    prob_dying: float,
    prob_spread: float,
    *,
    verbose: bool = False
) -> float:
    # Convert probabilities using math.erf (for better optimization behaviour)
    prob_mutate = (1.0 + math.erf(prob_mutate)) / 2.0
    prob_dying = (1.0 + math.erf(prob_dying)) / 2.0
    prob_spread = (1.0 + math.erf(prob_spread)) / 2.0

    # Create model and simulation
    model = create_six_mutations_model(
        prob_mutate=prob_mutate,
        prob_dying=prob_dying,
        prob_spread=prob_spread
    )
    simulation = NeighboringCellMethod(model)

    # Assumption: number of cells
    num_cells = 1_000_000

    # Compute loss as total square difference of probabilities
    loss = 0.0
    prev_prob_cdf = 0.0
    for age, prob in control_group_distribution:
        prob_cdf = compute_cancer_probability(
            method=simulation,
            num_cells=num_cells,
            time=age * 12,  # 1 time step = 1 month
            state_cancer=7  # C
        )
        prob_est = prob_cdf - prev_prob_cdf
        loss += (prob_est - prob) ** 2

        if verbose:
            print(f"model({age}) = {prob_cdf - prev_prob_cdf} (expected {prob})")

        prev_prob_cdf = prob_cdf

    if verbose:
        print()
        print(f"prob_mutate = {prob_mutate}")
        print(f"prob_dying  = {prob_dying}")
        print(f"prob_spread = {prob_spread}")
        print()
        print(f"loss = {loss}")

    return loss

In [ ]:
# Optimize parameters for toy model
if 'params_two_states' not in locals():
    params_two_states = {
        'prob_mutate': -4.0,
        'prob_spread': -4.0
    }

params_two_states = steepest_descent(
    loss_two_state_model,
    params_two_states,
    num_iter=1000,
    delta=0.01
)

print()
print(f"┌────────────────────┐")
print(f"│ PRINTING ESTIMATES │")
print(f"└────────────────────┘")
loss_two_state_model(**params_two_states, verbose=True);

In [ ]:
# Optimize parameters for model
if 'params_six_mutations' not in locals():
    params_six_mutations = {
        'prob_mutate': -1.446992780508533,
        'prob_dying': 0.11531864085930957,
        'prob_spread': -1.5174988338876292
    } # loss = 1.95e-6

params_six_mutations = steepest_descent(
    loss_six_mutations_model,
    params_six_mutations,
    num_iter=100,
    delta=0.001
)

print()
print(f"┌────────────────────┐")
print(f"│ PRINTING ESTIMATES │")
print(f"└────────────────────┘")
loss_six_mutations_model(**params_six_mutations, verbose=True)

In [ ]:
print_latex_table(
    model = create_six_mutations_model(
        prob_mutate = 0.020360638820134225,
        prob_dying  = 0.5647743182095744,
        prob_spread = 0.01593379960021246
    ),
    method = NeighboringCellMethod,
    num_cells = 1_000_000,
    state_cancer = 7,
    distribution = control_group_distribution
)